In [1]:
import os
import pandas as pd
from quant_utils.constant_varialbles import LAST_TRADE_DT
from openpyxl import load_workbook

from openpyxl.styles import Color, Font, Alignment, PatternFill, Side, Border
import emoji
import quant_utils.data_moudle as dm
from quant_utils.db_conn import DB_CONN_JJTG_DATA
from quant_utils.utils import save_excel_into_img
from quant_utils.xlsx_func import XlsxFunc

from watermarker.marker import add_mark
from wrapper.wx_wrapper import WxWrapper


exec_query耗时:0.0150 s

exec_query耗时:0.0980 s


1日报

In [2]:

template_path = "E:/基金投顾自动化/"
month = LAST_TRADE_DT[4:6]
# 读取模板文件
df1 = pd.read_excel(
    template_path + "基金投顾业务数据报表-t+1.xlsx", 
    sheet_name="分公司客户购买情况-报表2_分公司客户购买情况",
)
df1["所属一级分公司"] = df1["所属一级分公司"].ffill()
df1 = df1.groupby(by=["所属一级分公司"]).sum().reset_index()

df2 = pd.read_excel(
    template_path + "基金投顾业务数据报表-t+1.xlsx", 
    sheet_name="分公司维度统计情况（剔除已解约客户）-报表2_分公司购买情",
)
df2["所属一级分公司"] = df2["所属一级分公司"].ffill()
df2 = df2.groupby(by=["所属一级分公司"]).sum()["保有客户数"].reset_index()

company_type = pd.read_excel(template_path+"艾特名单.xlsx")
df2 = df2[["所属一级分公司", "保有客户数"]]
df1 = (
    df1
    .merge(df2, on="所属一级分公司", how="left")
    .merge(company_type, left_on="所属一级分公司", right_on="分公司", how="left")
)


d:\miniforge3\envs\my312\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
d:\miniforge3\envs\my312\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [3]:
col = [
    "分公司类别FLAG",
    "分公司类别", 
    "所属一级分公司", 
    "客户基金投顾总资产-月日均 服务关系2(万)", 
    "客户基金投顾总资产 服务关系2(万)", 
    "保有客户数", 
    "当月基金投顾购买金额-服务关系2(万)", 
    "当月基金投顾客户数", 
    "本月累计赎回到账金额-服务关系2(万)"
]
result = df1[col]
rename_dict = {
    "所属一级分公司": "分公司", 
    "客户基金投顾总资产-月日均 服务关系2(万)": "月日均规模(万元)", 
    "客户基金投顾总资产 服务关系2(万)": "保有规模(万元)", 
    "保有客户数": "存续客户数", 
    "当月基金投顾购买金额-服务关系2(万)": f"{month}月新增签约资产(万元)", 
    "当月基金投顾客户数": f"{month}月新增签约客户数", 
    "本月累计赎回到账金额-服务关系2(万)": f"{month}月赎回到账金额(万元)"
}
result["本月累计赎回到账金额-服务关系2(万)"] = - result["本月累计赎回到账金额-服务关系2(万)"]
result.rename(columns=rename_dict, inplace=True)

result.sort_values(
    by=["分公司类别FLAG", "月日均规模(万元)"], 
    ascending=[True, False],
    inplace=True
)
result.fillna(0, inplace=True)
result.index = range(0, result.shape[0])

C:\Users\Wilcoxon\AppData\Local\Temp\ipykernel_8764\2794208013.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result["本月累计赎回到账金额-服务关系2(万)"] = - result["本月累计赎回到账金额-服务关系2(万)"]
C:\Users\Wilcoxon\AppData\Local\Temp\ipykernel_8764\2794208013.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result.rename(columns=rename_dict, inplace=True)
C:\Users\Wilcoxon\AppData\Local\Temp\ipykernel_8764\2794208013.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.or

In [4]:
def get_company_str(df, fund_type, order_column):
    temp1 = df.query(f"分公司类别=='{fund_type}'")
    temp1 = temp1[temp1[order_column]>0]
    temp1["rank"] = temp1[order_column].rank(ascending=False, method='min')
    temp1.sort_values(by="rank", ascending=True, inplace=True)
    temp1 = temp1.query("rank<3")[["分公司", "rank"]]
    rank_list = temp1["rank"].unique()
    result_list = []
    for rank in rank_list:
        rank_result = temp1.query(f"rank=={rank}")
        result_list.append("/".join(rank_result["分公司"]))
    return '、'.join(result_list)

In [5]:
new_client_num = result.loc[result.shape[0]-1, f"{month}月新增签约客户数"]
new_client_asset = result.loc[result.shape[0]-1, f"{month}月新增签约资产(万元)"]
total_client_num = result.loc[result.shape[0]-1, "存续客户数"]
total_client_asset = result.loc[result.shape[0]-1, "保有规模(万元)"]

In [6]:

xlsx_func = XlsxFunc(file_path = f"{template_path}日报周报/模板/基金投顾日报.xlsx")
xlsx_func.set_value(
    cell_string="A1",
    value=(
        f"基金投顾业务{month}月-日报({LAST_TRADE_DT[4:]})" + "\n"
        + f"本月新增签约客户数{new_client_num:.0f}位,本月新增签约资产{new_client_asset:.0f}万元"+"\n"
        + f"存续客户数{total_client_num:.0f}位,保有规模{total_client_asset/10000:.2f}亿元"
    )
)

for row in range(3, xlsx_func.max_row + 1):
    for col in range(2, xlsx_func.max_column + 1):
        xlsx_func.set_value(
            row=row, column=col, value=result.iloc[row-3, col]
        )
xlsx_func.save(
    new_path=f"{template_path}日报周报/结果/日报.xlsx"
)


新增客户分公司

In [7]:
company_type = result[["分公司类别FLAG", "分公司类别"]].drop_duplicates().sort_values("分公司类别FLAG")
company_type = company_type.query("分公司类别FLAG != 0")

In [8]:
company_type

,分公司类别FLAG,分公司类别
0,1.0,一类
9,2.0,二类
17,3.0,三类
27,4.0,四类
37,5.0,五类


In [9]:
new_client_dict = {
    type: get_company_str(
        df=result, fund_type=type, order_column=f"{month}月新增签约客户数"
    )
    for type in company_type["分公司类别"]
}
new_client = [f"{key}: {val}" for key, val in new_client_dict.items()]
new_client = ",\n".join(new_client)

新增规模分公司

In [10]:
new_asset_dict = {
    type: get_company_str(
        df=result, fund_type=type, order_column=f"{month}月新增签约资产(万元)"
    )
    for type in company_type["分公司类别"]
}
new_asset = [f"{key}: {val}" for key, val in new_asset_dict.items()]
new_asset = ",\n".join(new_asset)

In [11]:
daily_report_string = f"""{emoji.emojize("🌹")}【基金投顾业务日报{LAST_TRADE_DT[4:]}】
全司基金投顾存续客户{total_client_num:.0f}人，保有规模{total_client_asset/10000:.2f}亿元
{emoji.emojize("🌝")}本月新增客户前3:
""" + new_client + '\n' + f"""
{emoji.emojize("🌝")}本月新增规模前3:"""+ '\n' + new_asset + '\n' + """
*保有客户数口径定义:基金投顾客户为当月日均基金投顾产品保有规模大于0或近12个月内日均基金投顾产品保有规模大于5000元的客户
"""
print(daily_report_string)

🌹【基金投顾业务日报1218】
全司基金投顾存续客户16642人，保有规模4.46亿元
🌝本月新增客户前3:
一类: 泉州分公司、浙江分公司,
二类: 四川分公司、湖北分公司,
三类: 苏州分公司、云南分公司,
四类: 广西分公司、内蒙古分公司,
五类: 大连分公司、西藏分公司

🌝本月新增规模前3:
一类: 泉州分公司、江苏分公司,
二类: 四川分公司、湖北分公司,
三类: 云南分公司、莆田分公司,
四类: 广西分公司、黑龙江分公司,
五类: 大连分公司、西藏分公司

*保有客户数口径定义:基金投顾客户为当月日均基金投顾产品保有规模大于0或近12个月内日均基金投顾产品保有规模大于5000元的客户



2.周报

In [12]:
col = [
    "分公司类别FLAG",
    "分公司类别", 
    "所属一级分公司", 
    "当周基金投顾购买金额-服务关系2(万)", 
    "当周基金投顾客户数", 
    "本周累计赎回到账金额-服务关系2(万)", 
    "当年基金投顾购买金额-服务关系2(万)", 
    "客户基金投顾总资产 服务关系2(万)",
    "保有客户数"
]
order_col = [
    "分公司类别FLAG",
    "分公司类别", 
    "分公司", 
    "本周新增签约资产(万元)",
    "本周新增签约客户数",
    "本周赎回到账金额(万元)",
    "本周净新增(万元)",
    "本年新增签约资产(万元)",
    "保有规模-小福星未折算(万元)",
    "存续客户数"
]
week_result = df1[col]
week_rename_dict = {
    "所属一级分公司": "分公司",
    "当周基金投顾购买金额-服务关系2(万)": "本周新增签约资产(万元)",
    "当周基金投顾客户数": "本周新增签约客户数",
    "本周累计赎回到账金额-服务关系2(万)":"本周赎回到账金额(万元)",
    "当年基金投顾购买金额-服务关系2(万)":"本年新增签约资产(万元)",
    "客户基金投顾总资产 服务关系2(万)": '保有规模-小福星未折算(万元)',
    "保有客户数": "存续客户数",
}
week_result = week_result.rename(columns=week_rename_dict)
week_result["本周净新增(万元)"] = week_result["本周新增签约资产(万元)"] - week_result["本周赎回到账金额(万元)"]

week_result=week_result[order_col]
week_result.sort_values(by=["分公司类别FLAG","本周新增签约资产(万元)"], ascending=[True, False], inplace=True)
week_result.index= range(0,len(week_result))


In [13]:
new_client_num_week = week_result.loc[result.shape[0]-1, "本周新增签约客户数"]
new_client_asset_week = week_result.loc[result.shape[0]-1, "本周新增签约资产(万元)"]
total_client_asset_week = week_result.loc[result.shape[0]-1, "保有规模-小福星未折算(万元)"]


last_week = dm.offset_period_trade_dt(LAST_TRADE_DT,-1, "w")
week_start = dm.offset_trade_dt(last_week,-1)


exec_query耗时:0.1270 s

exec_query耗时:0.1010 s


In [14]:
xlsx_func = XlsxFunc(file_path = f"{template_path}日报周报/模板/基金投顾周报.xlsx")
xlsx_func.set_value(
    cell_string="A1",
    value= (
        f"基金投顾业务周报({week_start[4:]}-{LAST_TRADE_DT[4:]})" + "\n"
        + f"本周新增签约资产{new_client_asset_week:.0f}万元,本周新增签约客户数{new_client_num_week:.0f}个"+"\n"
    )
)

for row in range(3, xlsx_func.max_row + 1):
    for col in range(2, xlsx_func.max_column + 1):
        xlsx_func.set_value(
            row=row, column=col, value=week_result.iloc[row-3, col]
        )
xlsx_func.save(
    new_path=f"{template_path}日报周报/结果/周报.xlsx"
)

周度金额

In [15]:
new_asset_dict = {
    type: get_company_str(
        df=week_result, fund_type=type, order_column="本周新增签约资产(万元)"
    )
    for type in company_type["分公司类别"]
}
new_asset = [f"{key}: {val}" for key, val in new_asset_dict.items()]
new_asset = ",\n".join(new_asset)

周度赎回金额


In [16]:
new_redeem_dict = {
    type: get_company_str(
        df=week_result, fund_type=type, order_column="本周赎回到账金额(万元)"
    )
    for type in company_type["分公司类别"]
}
new_redeem = [f"{key}: {val}" for key, val in new_redeem_dict.items()]
new_redeem = ",\n".join(new_redeem)

保有规模

In [17]:
new_total_dict = {
    type: get_company_str(
        df=week_result, fund_type=type, order_column="保有规模-小福星未折算(万元)"
    )
    for type in company_type["分公司类别"]
}
new_total = [f"{key}: {val}" for key, val in new_total_dict.items()]
new_total = ",\n".join(new_total)

In [18]:
week_report_string = f"""{emoji.emojize("🌟")}基金投业务周报【{week_start[4:]}-{LAST_TRADE_DT[4:]}】
{emoji.emojize("✊")}周度新增前3:
"""+ new_asset + '\n' + f"""
{emoji.emojize("💔")}周度赎回金额前3:
""" + new_redeem + '\n' + f"""
{emoji.emojize("💪")}保有规模前3:
"""+ new_total + '\n'
print(week_report_string)

🌟基金投业务周报【1212-1218】
✊周度新增前3:
一类: 广东分公司、泉州分公司,
二类: 湖北分公司、南平分公司/漳州分公司,
三类: 莆田分公司、苏州分公司,
四类: 广西分公司、山西分公司,
五类: 大连分公司

💔周度赎回金额前3:
一类: 泉州分公司、福州分公司,
二类: 南平分公司、漳州分公司,
三类: 河南分公司、宁波分公司,
四类: 内蒙古分公司、山西分公司,
五类: 大连分公司、新疆分公司

💪保有规模前3:
一类: 泉州分公司、广东分公司,
二类: 湖北分公司、四川分公司,
三类: 云南分公司、安徽分公司,
四类: 广西分公司、内蒙古分公司,
五类: 西藏分公司、大连分公司



目标盈周报

In [19]:
query_sql = """
SELECT
	a.*,
	b.`累计收益率(止盈后)`,
	b.`年化收益率(止盈后)`,
	c.PORTFOLIO_VALUE AS '最大回撤',
	d.TEMPERATURE AS '权益温度',
	e.`债市温度计` 
FROM
	`view_mointor_target_portfolio` a
	JOIN view_mointor_target_portfolio_accumulated b ON a.`组合名称` = b.`组合名称`
	LEFT JOIN portfolio_derivatives_performance c ON c.END_DATE = b.`交易日` 
	AND c.TICKER_SYMBOL = a.`组合名称` 
	AND c.`CYCLE` = '成立日' 
	AND c.`INDICATOR` = '最大回撤'
	JOIN view_temperature_stock d ON d.END_DATE = a.`运作起始日` 
	AND d.TICKER_SYMBOL = '000985'
	JOIN view_temperature_bond e ON e.`日期` = d.END_DATE
	JOIN portfolio_info f ON f.PORTFOLIO_NAME = a.`组合名称` 
WHERE
	1 = 1 
	AND f.ORDER_ID BETWEEN 4.01 
	AND 5.99 
ORDER BY
	f.ORDER_ID
"""
target_df = DB_CONN_JJTG_DATA.exec_query(query_sql)
alive_port = target_df[pd.isnull(target_df["运营结束日期"])]
dead_port = target_df[~pd.isnull(target_df["运营结束日期"])]
target_df = pd.concat([dead_port, alive_port])
target_df.index = range(1, len(target_df)+1)
target_df.reset_index(inplace=True)
target_df.rename(columns={"index":"序号"}, inplace=True)
target_df["目标收益"] = target_df["目标收益"].str.split("%", expand=True)[0].astype(float)/100
target_df["达成收益率"] = target_df["达成收益率"].str.split("%", expand=True)[0].astype(float)/100
target_df["累计收益率"] = target_df["累计收益率"].str.split("%", expand=True)[0].astype(float)/100


exec_query耗时:4.7798 s


In [20]:
target_df["最大回撤"].fillna("--", inplace=True)
dead_port = target_df[~pd.isnull(target_df["运营结束日期"])]
alive_port = target_df[pd.isnull(target_df["运营结束日期"])]
needed_col = [
    "序号",
    "组合名称",
    "运作起始日",
    "交易日",
    "运作天数",
    "目标收益",
    "达成收益率",
    "累计收益率",
    "最大回撤",
    "权益温度",
    "债市温度计"
]
target_df = target_df[needed_col]
dead_port = dead_port[needed_col]
alive_port = alive_port[needed_col]

C:\Users\Wilcoxon\AppData\Local\Temp\ipykernel_8764\3383352734.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  target_df["最大回撤"].fillna("--", inplace=True)
C:\Users\Wilcoxon\AppData\Local\Temp\ipykernel_8764\3383352734.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '--' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  target_df["最大回撤"].fillna("--", inplace=True)


In [21]:

xlsx_func = XlsxFunc(
    file_path=f"{template_path}日报周报/模板/目标盈周报模板.xlsx"
)

xlsx_func.delete_rows(
    4,
    end_row=xlsx_func.max_row+1
)
xlsx_func.unmerge_all_cells()
xlsx_func.merge_cells(
    start_row=1, start_column=1,
    end_row=1, end_column=xlsx_func.max_column
)

xlsx_func.set_value(
    cell_string="A1",
    value=(
        f"知己目标盈\n(数据截止{LAST_TRADE_DT[:4]}年{LAST_TRADE_DT[4:6]}月{LAST_TRADE_DT[6:]}日)"
    )
)

for row in range(3, dead_port.shape[0] + 3):
    for col in range(2, xlsx_func.max_column + 1):
        xlsx_func.set_value(
            row=row,
            column=col,
            value=dead_port.iloc[row-3, col-2]
        )

xlsx_func.set_value(
    row=dead_port.shape[0] + 3, column=9, value=dead_port["累计收益率"].mean()
)
xlsx_func.set_value(
    row=dead_port.shape[0] + 3, column=8, value=dead_port["达成收益率"].mean()   
)
xlsx_func.set_value(
    row=dead_port.shape[0] + 3, column=6, value=dead_port["运作天数"].mean()
)
xlsx_func.set_value(
    row=dead_port.shape[0] + 3, column=11, value=dead_port["权益温度"].mean()
)
xlsx_func.set_value(
    row=dead_port.shape[0] + 3, column=12, value=dead_port["债市温度计"].mean()
)

xlsx_func.merge_cells(
    start_row=3, start_column=1, 
    end_row=dead_port.shape[0] + 3, end_column=1
)
xlsx_func.set_value(
    row=3, column=1, value="已止盈"
)
xlsx_func.set_alignment(
    row=3, 
    column=1, 
    alignment= Alignment(
        horizontal='center',
        vertical='center'
    )
)
xlsx_func.merge_cells(
    start_row=dead_port.shape[0] + 3, start_column=2, 
    end_row=dead_port.shape[0] + 3, end_column=5
)
xlsx_func.set_value(
    row=dead_port.shape[0] + 3, column=2, value="均值"
)


for row in range(dead_port.shape[0] + 4, target_df.shape[0] + 4):
    for col in range(2, xlsx_func.max_column + 1):
        xlsx_func.set_value(
            row=row,
            column=col,
            value=alive_port.iloc[row-dead_port.shape[0]-4, col-2]
        )

for col in range(1, xlsx_func.max_column + 1):
    for row in range(4, xlsx_func.max_row+  1):
        xlsx_func.format_painter(
            row=row,
            column=col,
            end_row=row,
            end_column=col,
            source_column=col,
            source_row=3,
            end_source_row=3,
            end_source_column=col
        )

xlsx_func.merge_cells(
    start_row=dead_port.shape[0] + 4, start_column=1, 
    end_row=target_df.shape[0] + 3, end_column=1
)

xlsx_func.set_value(
    row=dead_port.shape[0] + 4, column=1, value="运行中"
)

xlsx_func.set_alignment(
    row=dead_port.shape[0] + 4, 
    column=1, 
    alignment= Alignment(
        horizontal='center',
        vertical='center'
    )
)

for col in range(2, xlsx_func.max_column + 1):
    xlsx_func.set_font(
        row=dead_port.shape[0] + 3, column=col,
        font=Font(
            color="FF0000",
            name=u'微软雅黑', 
            bold = True,
            size = 11
        )
    )

xlsx_func.save(
    new_path=f"{template_path}日报周报/结果/目标盈周报结果.xlsx"
)

In [22]:
file_list = [
    "目标盈周报结果.xlsx",
    "日报.xlsx",
    "周报.xlsx",
]
for file in file_list:
    if file.endswith(".xlsx") and not file.startswith("~"):
        save_excel_into_img(
            file_path=f"E:/基金投顾自动化/日报周报/结果/{file}",
            img_name=f"E:/基金投顾自动化/结果/{file[:-5]}.png"
        )
        add_mark(
            file = f"E:/基金投顾自动化/结果/{file[:-5]}.png", 
            out = "E:/基金投顾自动化/结果/",
            mark = "仅限内部参考，不可对外发送!", 
            opacity=0.15, 
            angle=30, 
            space=100,
            size=50,
            color="#DB7093"
        )

E:/基金投顾自动化/结果/目标盈周报结果.png 保存成功。
E:/基金投顾自动化/结果/日报.png 保存成功。
E:/基金投顾自动化/结果/周报.png 保存成功。


In [23]:
robot = WxWrapper()
robot.send_image(
    image_path="E:/基金投顾自动化/结果/日报.png"
)
robot.send_text(
    content=daily_report_string
)
mentioned_mobile_list = robot.get_mentioned_moble_list_by_name(
    ["陆天琦"]
)


robot.send_text(
    content="业务日报来喽,请及时发送",
    mentioned_mobile_list=mentioned_mobile_list
)

if dm.if_period_end(LAST_TRADE_DT) ==1:
    robot.send_image(
        image_path="E:/基金投顾自动化/结果/目标盈周报结果.png"
    )
    mentioned_mobile_list = robot.get_mentioned_moble_list_by_name(
        ["程钰杨"]
    )
    robot.send_text(
        content="目标盈周报来喽,请及时发送",
        mentioned_mobile_list=mentioned_mobile_list
    )
    robot.send_image(
        image_path="E:/基金投顾自动化/结果/周报.png"
    )

    robot.send_text(
        content=week_report_string
    )
    mentioned_mobile_list = robot.get_mentioned_moble_list_by_name(
        ["陆天琦"]
    )
    robot.send_text(
        content="业务周报来喽,请及时发送",
        mentioned_mobile_list=mentioned_mobile_list
    )

消息发送成功
消息发送成功
消息发送成功

exec_query耗时:0.0020 s
